In [17]:
import sys
sys.path.append('../_amigocloud')
from amigocloud import AmigoCloud
import pandas as pd
from shapely import wkb
import geopandas as gpd
from shapely.geometry import Point, Polygon, MultiPolygon, MultiPoint
import os

In [18]:
def get_unidad_local():
    try:
        # ruta del archivo que contiene la información necesaria
        with open('../../unidad_local.txt', 'r') as file:
            unidad = file.readline().strip()
            return unidad
    except FileNotFoundError:
        print("El archivo 'unidad_local.txt' no se encuentra en la ruta especificada.")
    return None

def get_ruta_local():
    try:
        with open('../ruta_local.txt', 'r') as file:
            ruta = file.readline().strip()
            return ruta
    except FileNotFoundError:
        print("El archivo 'ruta_local.txt' no se encuentra en la ruta especificada.")
    return None

In [20]:
# RUTA ONEDRIVE
unidad = get_unidad_local()
ruta = get_ruta_local()
ruta_completa = os.path.join(unidad, ruta)
ruta_completa

'G:\\OneDrive - Ingenio Azucarero Guabira S.A\\_DATOS_PYTHON'

In [21]:
# leer token
acceso = open(ruta_completa + '/_keys/api_amigocloud_gral.key','r')
api_token = acceso.readlines()[0]
acceso.close()
amigocloud = AmigoCloud(token=api_token)
amigocloud

In [22]:
path_xlsx_parte = unidad + r'\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - EQUIPO AVIACION UTEA\Pulverizacion\2025\parte_diario_pulverizacion.xlsx'

In [23]:
consulta = {'query': 'select * from dataset_367119'}
seleccion = amigocloud.get('https://app.amigocloud.com/api/v1/projects/35248/sql', consulta)
datos = seleccion['data']

In [24]:
df_datos = gpd.GeoDataFrame(datos)

In [25]:
df_datos

,id,amigo_id,fecha_registro,referencia,temp_max,temp_min,viento,humedad,observaciones,tipo_cultivo,tipo_desarrollo
0,24,290c20acd6444b03993f23ab86811205,2025-03-28 19:53:47+00:00,0101000020E6100000CB80B3942C974FC07164D46B6948...,29.0,28.0,1.2,80.0,None,CAÑA DE AZUCAR,MADURACION


In [26]:
list(set(df_datos['id']))

[24]

In [27]:
idd = 24

In [28]:
query = {'query': f'SELECT apli.*, plan.unidad_01, plan.unidad_03, plan.os\
    FROM dataset_367119 apli\
    JOIN dataset_360912 plan\
    ON ST_Intersects(apli.referencia, plan.geometry)\
    where apli.id = {idd}'}
select = amigocloud.get('https://app.amigocloud.com/api/v1/projects/35248/sql', query)
data_apli = select['data']
#df_apli = pd.DataFrame(data_apli)
#df_apli

In [29]:
dic_apli = data_apli[0]
dic_apli

{'id': 24,
 'amigo_id': '290c20acd6444b03993f23ab86811205',
 'fecha_registro': '2025-03-28 19:53:47+00:00',
 'referencia': '0101000020E6100000CB80B3942C974FC07164D46B694831C0',
 'temp_max': 29.0,
 'temp_min': 28.0,
 'viento': 1.2,
 'humedad': 80.0,
 'observaciones': None,
 'tipo_cultivo': 'CAÑA DE AZUCAR',
 'tipo_desarrollo': 'MADURACION',
 'unidad_01': 256,
 'unidad_03': 75,
 'os': 11}

In [30]:
cod_prop = dic_apli['unidad_01']
cod_ca = dic_apli['unidad_03']
oss = dic_apli['os']

In [31]:
print('propiedad:', cod_prop)
print('cañero:', cod_ca)
print('os:', oss)

propiedad: 256
cañero: 75
os: 11


In [32]:
#nulos = dic_apli.isnull().sum()
#nulos

In [33]:
df_parte = pd.read_excel(path_xlsx_parte)
df_parte.head(3)

,Nº DE LABOR,FECHA,SEMANA,INSTITUCION,COD CANERO,NOMBRE CANERO,COD PROPIEDAD,NOMBRE PROPIEDAD,LOTE,HECTAREA,...,OS,TEMP MAX,TEMP MIN,TIPO CULTIVO,TIPO DESARROLLO,NUM VUELOS,HORAS VUELO,TOTAL CAUDAL,idd,AREA PULVERIZADO
0,PA-2025-1,2025-03-07,10.0,50,41594,AGROPECUARIA CAMPO DULCE S.R.L.,30,CAMPO DULCE,EP-L31,30.260942,...,1,25.0,20.0,CAÑA DE AZUCAR,MADURACION,104.0,15:03:42,2952.664,1.0,28.333271
1,PA-2025-1,2025-03-07,10.0,50,41594,AGROPECUARIA CAMPO DULCE S.R.L.,30,CAMPO DULCE,EP-L16,23.216246,...,1,25.0,20.0,CAÑA DE AZUCAR,MADURACION,104.0,15:03:42,2952.664,1.0,21.634272
2,PA-2025-1,2025-03-07,10.0,50,41594,AGROPECUARIA CAMPO DULCE S.R.L.,30,CAMPO DULCE,ER-L6,19.683505,...,1,25.0,20.0,CAÑA DE AZUCAR,MADURACION,104.0,15:03:42,2952.664,1.0,19.160233


In [34]:
df_parte['FECHA'] = pd.to_datetime(df_parte['FECHA'], errors='coerce')
df_parte['HORA DE INICIO'] = pd.to_datetime(df_parte['HORA DE INICIO'], errors='coerce')
df_parte['HORA FINAL'] = pd.to_datetime(df_parte['HORA FINAL'], errors='coerce')

In [35]:
df_parte.loc[(df_parte['COD PROPIEDAD'] == dic_apli['unidad_01']) & 
             (df_parte['COD CANERO'] == dic_apli['unidad_03']) & 
             (df_parte['OS'] == dic_apli['os']), 'idd'] = dic_apli['id']
df_parte.loc[(df_parte['COD PROPIEDAD'] == dic_apli['unidad_01']) & 
             (df_parte['COD CANERO'] == dic_apli['unidad_03']) & 
             (df_parte['OS'] == dic_apli['os']), 'TEMP MAX'] = dic_apli['temp_max']
df_parte.loc[(df_parte['COD PROPIEDAD'] == dic_apli['unidad_01']) & 
             (df_parte['COD CANERO'] == dic_apli['unidad_03']) & 
             (df_parte['OS'] == dic_apli['os']), 'TEMP MIN'] = dic_apli['temp_min']
df_parte.loc[(df_parte['COD PROPIEDAD'] == dic_apli['unidad_01']) & 
             (df_parte['COD CANERO'] == dic_apli['unidad_03']) & 
             (df_parte['OS'] == dic_apli['os']), 'VIENTO'] = dic_apli['viento']
df_parte.loc[(df_parte['COD PROPIEDAD'] == dic_apli['unidad_01']) & 
             (df_parte['COD CANERO'] == dic_apli['unidad_03']) & 
             (df_parte['OS'] == dic_apli['os']), 'HUMEDAD'] = dic_apli['humedad']
df_parte.loc[(df_parte['COD PROPIEDAD'] == dic_apli['unidad_01']) & 
             (df_parte['COD CANERO'] == dic_apli['unidad_03']) & 
             (df_parte['OS'] == dic_apli['os']), 'TIPO CULTIVO'] = dic_apli['tipo_cultivo']
df_parte.loc[(df_parte['COD PROPIEDAD'] == dic_apli['unidad_01']) & 
             (df_parte['COD CANERO'] == dic_apli['unidad_03']) & 
             (df_parte['OS'] == dic_apli['os']), 'TIPO DESARROLLO'] = dic_apli['tipo_desarrollo']

In [36]:
df_parte.head(3)

,Nº DE LABOR,FECHA,SEMANA,INSTITUCION,COD CANERO,NOMBRE CANERO,COD PROPIEDAD,NOMBRE PROPIEDAD,LOTE,HECTAREA,...,OS,TEMP MAX,TEMP MIN,TIPO CULTIVO,TIPO DESARROLLO,NUM VUELOS,HORAS VUELO,TOTAL CAUDAL,idd,AREA PULVERIZADO
0,PA-2025-1,2025-03-07,10.0,50,41594,AGROPECUARIA CAMPO DULCE S.R.L.,30,CAMPO DULCE,EP-L31,30.260942,...,1,25.0,20.0,CAÑA DE AZUCAR,MADURACION,104.0,15:03:42,2952.664,1.0,28.333271
1,PA-2025-1,2025-03-07,10.0,50,41594,AGROPECUARIA CAMPO DULCE S.R.L.,30,CAMPO DULCE,EP-L16,23.216246,...,1,25.0,20.0,CAÑA DE AZUCAR,MADURACION,104.0,15:03:42,2952.664,1.0,21.634272
2,PA-2025-1,2025-03-07,10.0,50,41594,AGROPECUARIA CAMPO DULCE S.R.L.,30,CAMPO DULCE,ER-L6,19.683505,...,1,25.0,20.0,CAÑA DE AZUCAR,MADURACION,104.0,15:03:42,2952.664,1.0,19.160233


In [37]:
output_path = path_xlsx_parte
df_parte.to_excel(output_path, index=False)

# HACER COPIA Y ELIMINAR PARTE DIARIO Y PLANIFICACION DE AMIGOCLOUD

In [38]:
idd

24

In [39]:
#copia y elimina los idd indicados del parte diario de amigocloud
id_proyecto = f'https://app.amigocloud.com/api/v1/projects/35248/sql'
query = f'INSERT INTO dataset_367584 (id, fecha_registro, temp_max, temp_min, viento, humedad, observaciones, tipo_cultivo, tipo_desarrollo, referencia)\
        SELECT id, fecha_registro, temp_max, temp_min, viento, humedad, observaciones, tipo_cultivo, tipo_desarrollo, referencia\
        FROM dataset_367119\
        WHERE id = {idd}'
query_sql = {'query': query}
post = amigocloud.post(id_proyecto, query_sql)
query = {'query': f'DELETE FROM dataset_367119 WHERE id = {idd}'}
ejecutar = amigocloud.post(id_proyecto, query)
print('se proceso IDD:', idd)

se proceso IDD: 24


In [40]:
#copia y elimina los id indicados de la planificacion semanal de amigocloud
id_proyecto = f'https://app.amigocloud.com/api/v1/projects/35248/sql'
query = f'INSERT INTO dataset_360915 (fecha_registro, id, unidad_01, unidad_02, unidad_03, unidad_04, unidad_05, area, os, geometry)\
            SELECT fecha_registro, id, unidad_01, unidad_02, unidad_03, unidad_04, unidad_05, area, os, geometry\
            FROM dataset_360912\
            WHERE unidad_01 = {cod_prop} and os = {oss}'
query_sql = {'query': query}
post = amigocloud.post(id_proyecto, query_sql)
query = {'query': f'DELETE FROM dataset_360912 WHERE unidad_01 = {cod_prop}'}
ejecutar = amigocloud.post(id_proyecto, query)
print('se proceso ID:', idd)

se proceso ID: 24


## Copiar ids de PARTE DIARIO a PLANIFICACION

In [41]:
path_planificacion = unidad + r'\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - EQUIPO AVIACION UTEA\Pulverizacion\2025\SHP\PLANIFICACION.shp'

In [42]:
gdf_plan = gpd.read_file(path_planificacion)

In [43]:
gdf_plan.loc[(gdf_plan['unidad_01'] == dic_apli['unidad_01']) & 
             (gdf_plan['unidad_03'] == dic_apli['unidad_03']) & 
             (gdf_plan['os'] == dic_apli['os']), 'idd'] = dic_apli['id']

In [44]:
gdf_plan

,unidad_01,unidad_02,unidad_03,unidad_04,unidad_05,area,os,soca,id,inst,idd,geometry
0,30,CAMPO DULCE,41594,AGROPECUARIA CAMPO DULCE S.R.L.,EP-L31,30.260942,1,2,1,50,1.0,"POLYGON ((504085.659 8094406.031, 503737.808 8..."
1,30,CAMPO DULCE,41594,AGROPECUARIA CAMPO DULCE S.R.L.,EP-L16,23.216246,1,4,2,50,1.0,"POLYGON ((504352.053 8094797.970, 504272.088 8..."
2,30,CAMPO DULCE,41594,AGROPECUARIA CAMPO DULCE S.R.L.,ER-L6,19.683505,1,5,3,50,1.0,"POLYGON ((503260.583 8094596.306, 503714.735 8..."
3,30,CAMPO DULCE,41594,AGROPECUARIA CAMPO DULCE S.R.L.,EP-L35,7.795719,1,4,4,50,1.0,"POLYGON ((504693.348 8094607.629, 504673.327 8..."
4,30,CAMPO DULCE,41594,AGROPECUARIA CAMPO DULCE S.R.L.,EP-L34,7.048921,1,5,5,50,1.0,"POLYGON ((504137.327 8094415.183, 504648.080 8..."
...,...,...,...,...,...,...,...,...,...,...,...,...
133,256,LOS ANDES--AGUILERA,75,AGUILERA TARADELLES NELSON MARIANO,LA20,7.890023,11,2,140,50,24.0,"POLYGON ((481237.012 8089847.643, 481636.666 8..."
134,256,LOS ANDES--AGUILERA,75,AGUILERA TARADELLES NELSON MARIANO,LA16,24.621769,11,2,141,50,24.0,"POLYGON ((480558.376 8088818.995, 481054.399 8..."
135,256,LOS ANDES--AGUILERA,75,AGUILERA TARADELLES NELSON MARIANO,LA24,13.205059,11,1,142,50,24.0,"POLYGON ((480301.359 8089838.440, 480241.822 8..."
136,256,LOS ANDES--AGUILERA,75,AGUILERA TARADELLES NELSON MARIANO,LA23,15.996730,11,1,143,50,24.0,"POLYGON ((481067.239 8090028.913, 480287.808 8..."


In [45]:
gdf_plan.dtypes

unidad_01       int64
unidad_02      object
unidad_03       int64
unidad_04      object
unidad_05      object
area          float64
os              int64
soca            int64
id              int64
inst            int64
idd           float64
geometry     geometry
dtype: object

In [46]:
gdf_plan.to_file(path_planificacion, driver='ESRI Shapefile')